### AUTH

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize(project='ee-USERNAME/PROJECT')

### **Geometries**

In [ ]:
import ee
import folium
from IPython.display import HTML

#Initialize the Earth Engine API
ee.Initialize()

#Define the study area

geometryPINEGULCH = ee.Geometry.Polygon(
        [[[-108.80164742558028, 39.28402267340665],
          [-108.23722481815841, 39.29358857295844],
          [-108.23997140018966, 39.56831366828538],
          [-108.79752755253341, 39.553491364366145]
          ]])

geometryGRIZZLYCREEK = ee.Geometry.Polygon(
        [[[-107.40066503796388, 39.69746161256133],
          [-107.39860510144044, 39.46619798632684],
          [-106.90834020886231, 39.46778823854969],
          [-106.96670507702638, 39.74763415825682],
          [-107.40272497448731, 39.73337830588799]
          ]])

geometryCHERRYCANYON = ee.Geometry.Polygon(
        [[[-103.53011965523325, 37.51499254964799],
          [-103.53011965523325, 37.336135915833296],
          [-103.33785891304575, 37.33176822282621],
          [-103.33785891304575, 37.519349567331446]]]);

geometrySILVERCREEK = ee.Geometry.Polygon(
        [[[-105.29672060223413, 37.6911297494477],
          [-105.29672060223413, 37.35348518722596],
          [-104.96850404949976, 37.35348518722596],
          [-104.96850404949976, 37.6911297494477]]]);
# 2017-07-19

geometryDECKER = ee.Geometry.Polygon(
        [[[-106.02963654395263, 38.479133428395656],
          [-106.02963654395263, 38.38392784402862],
          [-105.91977326270263, 38.38392784402862],
          [-105.91977326270263, 38.479133428395656]]]);

# 2019-09-08

geometryLAKECHRIST = ee.Geometry.Polygon(
        [[[-107.09745152809535, 39.473198422531205],
          [-107.09745152809535, 39.36445604809376],
          [-106.97797520973597, 39.36445604809376],
          [-106.97797520973597, 39.473198422531205]]]);
#2018-07-30

geometryPALATEAU = ee.Geometry.Polygon(
        [[[-108.55952873183622, 37.67620981411429],
          [-108.55952873183622, 37.534774582410634],
          [-108.39198722792997, 37.534774582410634],
          [-108.39198722792997, 37.67620981411429]]]);

geometryHI = ee.Geometry.Polygon(
        [[[-156.7579797108181, 21.083739928456865],
          [-156.7579797108181, 20.707830399739446],
          [-156.17982419323997, 20.707830399739446],
          [-156.17982419323997, 21.083739928456865]]]);

geometries_dict = {
    'PineGulch': geometryPINEGULCH,
    'GrizzlyCreek': geometryGRIZZLYCREEK,
    'CherryCanyon': geometryCHERRYCANYON,
    'SilverCreek': geometrySILVERCREEK,
    'Decker': geometryDECKER,
    'LakeChrist': geometryLAKECHRIST,
    'Palateau': geometryPALATEAU,
    'HI': geometryHI
}

geometries = [
    geometryPINEGULCH,
    geometryGRIZZLYCREEK,
    geometryCHERRYCANYON,
    geometrySILVERCREEK,
    geometryDECKER,
    geometryLAKECHRIST,
    geometryPALATEAU,
    geometryHI
]

geometryHI.getInfo()

In [ ]:
def get_fire_boundary(event_id):
    dataset = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')
    filtered_dataset = dataset.filter(ee.Filter.eq('Event_ID', event_id))

    if filtered_dataset.size().getInfo() > 0:
        selected_fire = ee.Feature(filtered_dataset.first())
        fire_geometry = selected_fire.geometry()
        geometry_type = fire_geometry.type().getInfo()

        if geometry_type == 'Polygon':
            coordinates = fire_geometry.coordinates().getInfo()
            fire_polygon = ee.Geometry.Polygon(coordinates)
            return fire_polygon
        elif geometry_type == 'GeometryCollection':
            geometries = fire_geometry.geometries().getInfo()
            fire_multipolygon = ee.Geometry.MultiPolygon(geometries)
            return fire_multipolygon
        else:
            print(f"Unsupported geometry type: {geometry_type}")
            return None
    else:
        print(f"No features found with the specified Event_ID: {event_id}")
        return None


event_id_PineGulch = 'CO3933610852620200731'
event_id_GrizzlyCreek = 'CO3957210726620200810'
event_id_CherryCanyon = 'CO3736710345020200520'
event_id_Decker = 'CO3840910600420190908'
event_id_LakeChrist = 'CO3937110704320180703'
event_id_Palateau = 'CO3765810847420180722'
event_id_SilverCreek = 'CO4022310665520180719'


event_id_dict = {
    'PineGulch': 'CO3933610852620200731',
    'GrizzlyCreek': 'CO3957210726620200810',
    'CherryCanyon': 'CO3736710345020200520',
    'Decker': 'CO3840910600420190908',
    'LakeChrist': 'CO3937110704320180703',
    'Palateau': 'CO3765810847420180722',
    'SilverCreek': 'CO4022310665520180719'
}

fire_boundaries = {}

#for fire_name, event_id in event_id_dict.items():
#    fire_boundary = get_fire_boundary(event_id)
#
#    if fire_boundary is not None:
#        fire_boundaries[fire_name] = fire_boundary
#        print(f"Added fire boundary for {fire_name} to the fire_boundaries dictionary:")
#        print(fire_boundaries[fire_name].getInfo())

for fire_name, polygon in geometries_dict.items():
    #fire_boundary = get_fire_boundary(polygon)

    if polygon is not None:
        fire_boundaries[fire_name] = polygon
        print(f"Added fire boundary for {fire_name} to the fire_boundaries dictionary:")
        print(fire_boundaries[fire_name].getInfo())

### **DATES**

In [ ]:
fire_dates = {
    'PineGulch': {
        'prefire_start': '2020-05-20',
        'prefire_end': '2020-06-18',
        'postfire_start': '2020-09-20',
        'postfire_end': '2020-10-28'
    },
    'GrizzlyCreek': {
        'prefire_start': '2020-05-20',
        'prefire_end': '2020-07-18',
        'postfire_start': '2020-09-06',
        'postfire_end': '2020-11-01'
    },
    'CherryCanyon':{
        'prefire_start': '2020-04-20',
        'prefire_end': '2020-05-15',
        'postfire_start': '2020-07-01',
        'postfire_end': '2020-08-15'
    },
    'SilverCreek': { #Done
        'prefire_start': '2021-06-01',
        'prefire_end': '2021-08-10',
        'postfire_start': '2019-05-10',
        'postfire_end': '2019-06-30'
    },
    'Decker': { #done
        'prefire_start': '2019-05-30',
        'prefire_end': '2019-07-25',
       'postfire_start': '2020-5-10',
       'postfire_end': '2020-6-20'
    },
    'LakeChrist': { # done
        'prefire_start': '2017-08-01',
        'prefire_end': '2017-10-15',
        'postfire_start': '2018-09-15',
        'postfire_end': '2018-10-30'
    },
    'Palateau': { #Done
        'prefire_start': '2018-05-10',
        'prefire_end': '2018-06-20',
        'postfire_start': '2019-05-10',
        'postfire_end': '2019-06-20'
    },
    'HI': { #Done
        'prefire_start': '2022-08-10',
        'prefire_end': '2022-09-20',
        'postfire_start': '2023-08-13',
        'postfire_end': '2023-09-20'
    }
}

### **Sentinel 2**

In [ ]:
def maskS2sr(image):
    cloudBitMask = ee.Number(2).pow(10).int()
    cirrusBitMask = ee.Number(2).pow(11).int()
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])

def create_rgb_image(image, geometry, viz_params={'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}):
    """Creates RGB image from the given image and geometry."""
    return image.clip(geometry).visualize(**viz_params)


def calculate_dnbr_and_classification(prefire_start, prefire_end, postfire_start, postfire_end, geometry):


    prefire_collection_S2 = (ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(geometry).filterDate(prefire_start, prefire_end).map(maskS2sr).median())
    print(prefire_collection_S2.getInfo())
    postfire_collection_S2 = (ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(geometry).filterDate(postfire_start, postfire_end).map(maskS2sr).median())
    print(postfire_collection_S2.getInfo())

    rgb_prefire_image = create_rgb_image(prefire_collection_S2, geometry)
    rgb_postfire_image = create_rgb_image(postfire_collection_S2, geometry)

    prefire_NBR_S2 = prefire_collection_S2.normalizedDifference(['B8', 'B12'])
    postfire_NBR_S2 = postfire_collection_S2.normalizedDifference(['B8', 'B12'])
    dNBR_unscaled_S2 = prefire_NBR_S2.subtract(postfire_NBR_S2)
    dNBR_S2 = dNBR_unscaled_S2.multiply(1000)

    thresholds_S2 = ee.Image([-1000, -251, -101, 99, 269, 439, 659, 2000])
    classified_S2 = dNBR_S2.lt(thresholds_S2).reduce('sum').toInt()

    return dNBR_S2, classified_S2, prefire_collection_S2, rgb_prefire_image, rgb_postfire_image


dnbr_layers = {}
classification_layers = {}
dnbr_and_classification_results = {}

#for fire_name, fire_boundary in fire_boundaries.items():
#    dates = fire_dates[fire_name]
#    prefire_start = dates["prefire_start"]
#    prefire_end = dates["prefire_end"]
#    postfire_start = dates["postfire_start"]
#    postfire_end = dates["postfire_end"]#
#
#    dnbr, classification, postfire_NBR_S2 = calculate_dnbr_and_classification(
#        prefire_start, prefire_end, postfire_start, postfire_end, fire_boundary
#    )#
#
#    dnbr_and_classification_results[fire_name] = {"dnbr": dnbr, "classification": classification, "NBR": postfire_NBR_S2}

for fire_name, polygon in geometries_dict.items():
    dates = fire_dates[fire_name]
    prefire_start = dates["prefire_start"]
    prefire_end = dates["prefire_end"]
    postfire_start = dates["postfire_start"]
    postfire_end = dates["postfire_end"]

    dnbr, classification, postfire_NBR_S2, rgb_prefire_image, rgb_postfire_image = calculate_dnbr_and_classification(
        prefire_start, prefire_end, postfire_start, postfire_end, polygon
    )

    dnbr_and_classification_results[fire_name] = {
        "dnbr": dnbr,
        "classification": classification,
        "NBR": postfire_NBR_S2,
        "rgb_prefire_image": rgb_prefire_image,
        "rgb_postfire_image": rgb_postfire_image
    }


print(dnbr_and_classification_results['HI']["dnbr"].getInfo())
print(dnbr_and_classification_results['HI']["classification"].getInfo())

#prefire_collection_S2 = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(fire_boundaries['']).filterDate('2018-05-05', '2018-06-26').map(maskS2sr)
#postfire_collection_S2 = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(fire_boundaries['']).filterDate('2018-09-15', '2018-10-29').map(maskS2sr)

#print("Number of prefire images:", prefire_collection_S2.size().getInfo())
#print("Number of postfire images:", postfire_collection_S2.size().getInfo())

In [ ]:
sld_intervals = ('<RasterSymbolizer>' +
'<ColorMap type="intervals" extended="false" >' +
'<ColorMapEntry color="#ffffff" quantity="-500" label="-500"/>' +
'<ColorMapEntry color="#7a8737" quantity="-250" label="-250" />' +
'<ColorMapEntry color="#acbe4d" quantity="-100" label="-100" />' +
'<ColorMapEntry color="#0ae042" quantity="100" label="100" />' +
'<ColorMapEntry color="#fff70b" quantity="270" label="270" />' +
'<ColorMapEntry color="#ffaf38" quantity="440" label="440" />' +
'<ColorMapEntry color="#ff641b" quantity="660" label="660" />' +
'<ColorMapEntry color="#a41fd6" quantity="2000" label="2000" />' +
'</ColorMap>' +
'</RasterSymbolizer>')

fire_names = [
    'PineGulch',
    'GrizzlyCreek',
    'CherryCanyon',
    'Decker',
    'LakeChrist',
    'Palateau',
    'SilverCreek'
]

fire_name_s2 = fire_names[3]
dnbr_to_display = dnbr_and_classification_results[fire_name_s2]["dnbr"]
classification_to_display = dnbr_and_classification_results[fire_name_s2]["classification"]
nbr = dnbr_and_classification_results[fire_name_s2]["NBR"]

map_dNBR_S2 = folium.Map(location=[39.40, -108.5], zoom_start=4)
map_class_S2 = folium.Map(location=[39.40, -108.5], zoom_start=4)

folium.TileLayer(tiles=dnbr_to_display.getMapId()['tile_fetcher'].url_format, attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>', overlay=True, name='dNBR').add_to(map_dNBR_S2)
folium.TileLayer(tiles=dnbr_to_display.sldStyle(sld_intervals).getMapId()['tile_fetcher'].url_format, attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>', overlay=True, name='dNBR classified').add_to(map_class_S2)

html_dNBR_S2 = map_dNBR_S2._repr_html_()
html_class_S2 = map_class_S2._repr_html_()

# Combine the HTML codes into a single string
html_combined_S2 = f'<table><tr><td>{html_dNBR_S2}</td><td>{html_class_S2}</td></tr></table>'

# Display the maps using the combined HTML
HTML(html_combined_S2)

In [ ]:
rgb_prefire_to_display = dnbr_and_classification_results[fire_name_s2]["rgb_prefire_image"]
rgb_postfire_to_display = dnbr_and_classification_results[fire_name_s2]["rgb_postfire_image"]

map_rgb_prefire = folium.Map(location=[39.40, -108.5], zoom_start=4)
map_rgb_postfire = folium.Map(location=[39.40, -108.5], zoom_start=4)

folium.TileLayer(
    tiles=rgb_prefire_to_display.getMapId()['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Prefire RGB'
).add_to(map_rgb_prefire)

folium.TileLayer(
    tiles=rgb_postfire_to_display.getMapId()['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Postfire RGB'
).add_to(map_rgb_postfire)

# Convert the maps to HTML.
html_rgb_prefire = map_rgb_prefire._repr_html_()
html_rgb_postfire = map_rgb_postfire._repr_html_()

# Combine the HTML codes into a single string
html_combined_RGB = f'<table><tr><td>{html_rgb_prefire}</td><td>{html_rgb_postfire}</td></tr></table>'

# Display the maps using the combined HTML
HTML(html_combined_RGB)

### **Export S2** - Do Not Run Every Time. Only when downloading specific captures from S2.

In [ ]:
#export S2 BW image
#fire_name = 'PineGulch'
fire_name_s2 = fire_names[0]
dnbr_s2 = dnbr_and_classification_results[fire_name_s2]["dnbr"]
classification_s2 = dnbr_and_classification_results[fire_name_s2]["classification"]

image = dnbr_s2

# Set visualization parameters
viz_params = {
    'min': -1000,
    'max': 1000,
    'palette': ['white', 'black']
}

# Apply visualization parameters to the image
image = image.visualize(**viz_params)
output_path = f'S2_dNBR_{fire_name_s2}.tif'
scale = 20

# Start the export task
task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'Export GeoTIFF for {fire_name_s2}',
        folder='Cleaned_Fire_images_TIFF',
        fileNamePrefix=output_path[:-4],
        #region = geometryPINEGULCH,
        region=fire_boundaries[fire_name_s2],
        scale=scale,
        crs='EPSG:4326'
    )


task.start()

print(f"S2 Export task started for {fire_name_s2}")